In [2]:
import numpy as np 
import tensorflow as tf

In [34]:
def build_text_line_reader(filenames=None, shuffle=False, batch_size=1):
    filenames = tf.placeholder(tf.string, shape=[None])
    dataset = tf.data.TextLineDataset(filenames)
    dataset = dataset.map(lambda string: tf.string_split([string]).values)
    dataset = dataset.map(lambda string: tf.string_to_number(string, out_type=tf.float32))
    if shuffle: dataset = dataset.shuffle(1000)
    if batch_size > 1: dataset = dataset.batch(100)
    iterator = dataset.make_initializable_iterator()
    next_elements = iterator.get_next()
    return filenames, iterator, next_elements

In [40]:
train_filenames, train_iterator, train_elements = build_text_line_reader(shuffle=True, batch_size=100)
sess = tf.Session()
sess.run(train_iterator.initializer, feed_dict={train_filenames:[r'../../data/x_1000_128.txt']})
for i in range(10): 
    print(sess.run(train_elements).shape)

(100, 128)
(100, 128)
(100, 128)
(100, 128)
(100, 128)
(100, 128)
(100, 128)
(100, 128)
(100, 128)
(100, 128)


In [53]:
def build_depict_graph(inputs, kernel_shape, bias_shape):
    weights = tf.get_variable("weights", kernel_shape,
        initializer=tf.random_normal_initializer())
    biases = tf.get_variable("biases", bias_shape,
        initializer=tf.constant_initializer(0.0))
    weighted_sum = tf.add(tf.matmul(inputs, weights), biases)
    return tf.nn.softmax(weighted_sum)

In [54]:
def build_train_graph(defalut, input_dim, output_dim, func=''):
    inputs = tf.placeholder_with_default(defalut, shape=[None, output_dim], name='train_input')
    with tf.variable_scope('depict', reuse=False):
        graph = build_depict_graph(inputs, [input_dim, output_dim], [output_dim])
    with tf.variable_scope('target_distribution'):
        P = graph
        N = tf.reshape(tf.reduce_sum(P, axis=0), (-1, output_dim))
        N = tf.div(P, tf.pow(N, 0.5))
        D = tf.reshape(tf.reduce_sum(N, 1), (-1, 1))
        Q = tf.div(N, D)
    with tf.variable_scope('regularization'):
        prior = [1 / float(output_dim)] * output_dim
        U = tf.reshape(tf.constant(prior), (-1, output_dim))
        F = tf.reshape(tf.reduce_mean(Q, axis=0), (-1, output_dim))
    with tf.variable_scope('cost'):
        if func == 'func_04':
            with tf.variable_scope('func_04'):
                C = tf.multiply(Q, tf.log(tf.div(Q, P)))
                R = tf.multiply(Q, tf.log(tf.div(F, U)))
                L = tf.reshape(tf.reduce_sum(C + R, axis=1), (-1, 1))
                cost = tf.reduce_mean(L, name='cost')
        elif func == 'func_02':
            with tf.variable_scope('func_02'):
                C = tf.multiply(Q, tf.log(tf.div(Q, P)))
                L = tf.reshape(tf.reduce_sum(C, axis=1), (-1, 1))
                cost = tf.reduce_mean(L, name='cost')
        elif func == 'func_08':
            with tf.variable_scope('func_08'):
                C = -tf.multiply(Q, tf.log(P))
                L = tf.reshape(tf.reduce_sum(C, axis=1), (-1, 1))
                cost = tf.reduce_mean(L, name='cost')
        else:
            with tf.variable_scope('func_02'):
                C = tf.multiply(Q, tf.log(tf.div(Q, P)))
                L = tf.reshape(tf.reduce_sum(C, axis=1), (-1, 1))
                cost = tf.reduce_mean(L, name='cost')
    tf.summary.scalar('training cost', cost)
    optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)
    return inputs, optimizer

In [60]:
data_type = tf.float32
input_dim = 128
output_dim = 10
class flags(object):
    summaries_dir = './temp/logs/'
FLAGS = flags()

x = np.loadtxt('../../data/x_1000_128.txt')
x.shape

(1000, 128)

In [58]:
train_graph = tf.Graph()
with train_graph.as_default():
    train_filenames, train_iterator, train_elements = \
        build_text_line_reader(shuffle=True, batch_size=100)
    train_input, optimizer = build_train_graph(train_elements, input_dim, output_dim)
    initializer = tf.global_variables_initializer()

checkpoints_path = "./tmp/model/checkpoints"
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/train',
                                     sess.graph)

train_sess = tf.Session(graph=train_graph)

train_sess.run(initializer)
train_sess.run(train_iterator.initializer, feed_dict={train_filenames: '../../data/x_1000_128.txt'})
for i in itertools.count():
    train_summary, _ = train_sess.run([merged, optimizer])
    train_writer.add_summary(train_summary, i)

ValueError: Dimensions must be equal, but are 10 and 128 for 'depict/MatMul' (op: 'MatMul') with input shapes: [?,10], [128,10].